<a href="https://colab.research.google.com/github/mathewpolonsky/RuTube-NER/blob/main/Train_labse_and_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers seqeval razdel datasets accelerate

In [ ]:
!gdown 1-iGz2p8KRe3L4yUt7M2ic0j9QJNyhkeV

!gdown 1r3ZGoTperYSCmkRErjXtj6druEcEoQxc

!gdown 1x3nUwRFPeYx0FKfyuW58UWRmKltHPNmv

!gdown 1byXSGJmF1URwJXr31dU7g9HvZCtVlDhO

Downloading...
From: https://drive.google.com/uc?id=1r3ZGoTperYSCmkRErjXtj6druEcEoQxc
To: /content/sample_submission.csv
100% 2.38M/2.38M [00:00<00:00, 127MB/s]
Downloading...
From: https://drive.google.com/uc?id=1x3nUwRFPeYx0FKfyuW58UWRmKltHPNmv
To: /content/ner_data_train_weak_labeling_full_6000.csv
100% 9.35M/9.35M [00:00<00:00, 80.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1byXSGJmF1URwJXr31dU7g9HvZCtVlDhO
To: /content/ner_data_test.csv
100% 1.51M/1.51M [00:00<00:00, 115MB/s]


In [2]:
!gdown 1-8EjYzR6F_dYrPy7iyx3mGCfZF5HaRo8
!gdown 1-8YnePcic5PzH6Zej0ZGcjdbE_hBi4X7

Downloading...
From: https://drive.google.com/uc?id=1-8EjYzR6F_dYrPy7iyx3mGCfZF5HaRo8
To: /content/config.json
100% 3.94k/3.94k [00:00<00:00, 16.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-8YnePcic5PzH6Zej0ZGcjdbE_hBi4X7
To: /content/pytorch_model.bin
100% 709M/709M [00:11<00:00, 59.9MB/s]


In [3]:
!mkdir model

In [4]:
!mv pytorch_model.bin model/

In [11]:
!mv config.json model/

In [ ]:
!gdown 1-GVh6mRMrkyPrhuL7QvUNHK27GlQ0AZ2
!gdown 1-Jx9lzbctrscBA-dRAnNv1q2heX4W7II

Downloading...
From: https://drive.google.com/uc?id=1-GVh6mRMrkyPrhuL7QvUNHK27GlQ0AZ2
To: /content/config.json
100% 3.94k/3.94k [00:00<00:00, 16.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Jx9lzbctrscBA-dRAnNv1q2heX4W7II
To: /content/pytorch_model.bin
100% 709M/709M [00:13<00:00, 52.0MB/s]


In [ ]:
!mkdir wiki_model

In [ ]:
!mv /content/config.json wiki_model/
!mv /content/pytorch_model.bin wiki_model/

In [5]:
import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np
import pandas as pd
import razdel
from tqdm.autonotebook import tqdm

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

import pickle
import numpy as np
import transformers
from transformers import Trainer
from datasets import load_metric
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification

In [12]:
model = AutoModelForTokenClassification.from_pretrained("model").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner", device="cuda")

In [ ]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его
# тег (label) из разметки, чтобы потом предать в модель классификации токенов

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные
    # для нас числа - начало и конец слова в токенах

    raw_toks = list(razdel.tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## Preparing data

In [ ]:
data = pd.read_csv("ner_data_train_weak_labeling_full_6000.csv")
data.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."
2,<НАЗВАНИЕ:> 6 марта 2022 г. Каратэ шотокан. Кр...,"{""label"":""Дата""\,""offset"":12\,""length"":14\,""se..."
3,<НАЗВАНИЕ:> Международный Проект «Победа без г...,"{""label"":""название проекта""\,""offset"":34\,""len..."
4,<НАЗВАНИЕ:> Подготовка к отопительному сезону:...,"{""label"":""локация""\,""offset"":49\,""length"":10\,..."


In [ ]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать,
# удалить лишние '\' например, преобразовать из str в список dict-ов
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

df.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"[{'label': 'локация', 'offset': 26, 'length': ..."
1,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","[{'label': 'команда', 'offset': 13, 'length': ..."
2,<НАЗВАНИЕ:> 6 марта 2022 г. Каратэ шотокан. Кр...,"[{'label': 'Дата', 'offset': 12, 'length': 14,..."
3,<НАЗВАНИЕ:> Международный Проект «Победа без г...,"[{'label': 'название проекта', 'offset': 34, '..."
4,<НАЗВАНИЕ:> Подготовка к отопительному сезону:...,"[{'label': 'локация', 'offset': 49, 'length': ..."


In [ ]:
for text, ents in zip(df["video_info"], df["entities"]):
    if type(ents) is dict:
        continue

    for ent in ents:
        if ent["label"] in ['сезон', 'серия']:
            print(text)
            print(ent)
            break

<НАЗВАНИЕ:> Что делать женщине, если у неё два любовника, а выбрать нужно одного (2022) 1,2,3,4,5,6,7,8 серия <ОПИСАНИЕ:>Анна — успешный адвокат, которая не стесняясь говорит своей давней подруге Ларисе, что хочет выбрать «мужа на три месяца», так как у нее сложный проект и нужен громоотвод и постоянный секс. В финал ее отбора вышли два идеальных варианта: молодой технократ педант и возрастной бизнесмен гуляка. Оба успешны, оба интересны, и оба ею увлечены. Однако, неожиданно Анна узнаёт, что жизнь ее заиграет новыми и не факт, что яркими красками. Она станет бабушкой. Сын оболтус возвращается из Англии с невесткой пакистанкой. А значит теперь Анна выбирает не «мужа на три месяца», а дедушку своей внучке.
{'label': 'серия', 'offset': 88, 'length': 1, 'segment': '1'}
<НАЗВАНИЕ:> Клип на дораму ❣ "Цветочная команда: Брачное агентство Чосона" ❣ (Гэ Дон и Ма Хун) <ОПИСАНИЕ:>16 серий по 60мин (2019г) Южная Корея Ли Су (Со Чжи Хун) был обычным кузнецом в сельской местности, но в 23 года неож

In [ ]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его
# тег (label) из разметки, чтобы потом предать в модель классификации токенов

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные
    # для нас числа - начало и конец слова в токенах

    raw_toks = list(razdel.tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

### Making train ant test

In [ ]:
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.01, random_state=1)

In [ ]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [ ]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5698
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 58
    })
})

## Preprocess data

In [ ]:
model_checkpoint = "cointegrated/LaBSE-en-ru"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5698 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 5698
})

## Init Training pipeline

In [ ]:
# with open('id_to_label_map.pickle', 'rb') as f:
#     map_id_to_label = pickle.load(f)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
id2label = dict(enumerate(label_list))
label2id = {v: k for k, v in id2label.items()}

# data_collator = DataCollatorForTokenClassification(tokenizer)

# id2label = {str(k): v for k, v in map_id_to_label.items()}
# label2id = {v: k for k, v in id2label.items()}
# label_names = list(id2label.values())

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/LaBSE-en-ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

27

In [ ]:
args = TrainingArguments(
    "LaBSE_ner_nerel",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=6,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size = 32 ## depending on the total volume of memory of your GPU
)

## Train model

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
## for compute_metrics function
metric = load_metric("seqeval")

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.364169,0.507020,0.385071,0.437710,0.881910
2,No log,0.326393,0.537757,0.556872,0.547148,0.900858
3,0.374800,0.318092,0.565817,0.631517,0.596865,0.902534
4,0.374800,0.317053,0.561361,0.547393,0.554289,0.898346
5,0.374800,0.314534,0.579070,0.590047,0.584507,0.902743
6,0.258000,0.317431,0.566082,0.573460,0.569747,0.900440


TrainOutput(global_step=1074, training_loss=0.3113911484872829, metrics={'train_runtime': 507.2992, 'train_samples_per_second': 67.392, 'train_steps_per_second': 2.117, 'total_flos': 5856143617014372.0, 'train_loss': 0.3113911484872829, 'epoch': 6.0})

In [ ]:
trainer.save_model("LaBSE_nerel_last_checkpoint")

In [ ]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

NameError: name 'true_labels' is not defined

In [ ]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']

In [13]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cuda')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:
            print(f'{t:15s} {label_list[idx]:10s}')
    return labels

# submission

In [ ]:
from tqdm.notebook import tqdm

import pandas as pd

from razdel import tokenize

In [14]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cuda')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:
            print(f'{t:15s} {label_list[idx]:10s}')
    return labels

In [ ]:
test = pd.read_csv("ner_data_test.csv")
test.head(5)

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...",NaN
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,NaN
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,NaN
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,NaN
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,NaN


In [ ]:
test.shape

(1606, 2)

In [ ]:
test = test.astype(object)

In [ ]:
sample = pd.read_csv("sample_submission.csv")
sample_len = sample["entities_prediction"].apply(lambda x: len(eval(x)))

for i, elem in tqdm(enumerate(test["video_info"]), total=len(test["video_info"])):
    raw_toks = list(tokenize(elem))
    words = [tok.text for tok in raw_toks]

    labels = predict_ner(words, tokenizer, model, pipe, verbose=False)
    # submission.loc[i, 'video_info'] = elem["tokens"]

    labels = labels[1:-1]

    if len(labels) < sample_len[i]:
        amount = sample_len[i] - len(labels)
        labels += ['O'] * amount
        print(amount)

    labels = labels[:sample_len[i]]

    if len(labels) != sample_len[i]:
        print("Bad")

    # test.loc[i, 'entities_prediction'] = [[label] for label in labels]
    test.loc[i, 'entities_prediction'] = [[label] for label in labels]
# return submission

  0%|          | 0/1606 [00:00<?, ?it/s]

1
1
2
3


In [ ]:
len(labels), len(words)

(112, 112)

In [ ]:
len([tok.text for tok in list(tokenize(test["video_info"][0]))])

157

In [ ]:
len(test.loc[0]["entities_prediction"])

157

In [ ]:
print("\n".join([f"{text}\t\t{ent}" for (text, ent) in
                 zip([tok.text for tok in list(tokenize(test["video_info"][0]))],
                     test.loc[0]["entities_prediction"])]))

<		O
НАЗВАНИЕ		O
:		O
>		O
БОЕЦ		B-название проекта
БЕЗ		I-название проекта
ПРАВИЛ		I-название проекта
,		O
ТРЕЙЛЕР		O
на		O
русском		O
,		O
фильм		O
2021		B-Дата
|		O
боевик		O
,		O
MMA		O
<		O
ОПИСАНИЕ		O
:		O
>		O
Больше		O
информации		O
в		O
нашей		O
группе		O
ВК		O
:		O
<		O
LINK		O
>		O
Русский		O
трейлер		O
фильма		O
БОЕЦ		B-название проекта
БЕЗ		I-название проекта
ПРАВИЛ		I-название проекта
2021		B-Дата
г		I-Дата
🎬		O
Боец		B-название проекта
без		I-название проекта
правил		I-название проекта
(		O
2021		B-Дата
)		O
русский		O
трейлер		O
Notorious		B-название проекта
Nick		I-название проекта
(		O
2021		B-Дата
)		O
/		O
США		B-локация
драма		O
,		O
боевик		O
Дата		O
выхода		O
:		O
7		B-Дата
августа		I-Дата
2021		I-Дата
г		I-Дата
(		O
мир		O
)		O
2		B-Дата
сентября		I-Дата
2021		I-Дата
г		I-Дата
(		O
Россия		B-локация
)		O
Ник		B-персона
Ньюэлл		I-персона
,		O
однорукий		O
боец		O
ММА		O
,		O
получает		O
редкую		O
возможность		O
принять		O
участие		O
в		O
чемпионате		O
.		O
Герой	

In [ ]:
test["entities_prediction"]

0       [O, O, O, O, B-название проекта, I-название пр...
1       [O, O, O, O, O, O, O, O, O, O, O, O, B-видеоиг...
2       [O, O, O, O, O, O, B-видеоигра, I-видеоигра, O...
3       [O, O, O, O, B-персона, I-персона, O, O, O, O,...
4       [O, O, O, O, B-организация, I-организация, O, ...
                              ...                        
1601    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
1602    [O, O, O, O, B-персона, I-персона, O, O, O, O,...
1603    [O, O, O, O, O, B-персона, I-персона, O, O, O,...
1604    [O, O, O, O, B-персона, I-персона, O, O, O, O,...
1605    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: entities_prediction, Length: 1606, dtype: object

In [ ]:
print(test["entities_prediction"].apply(len).tolist())

[157, 165, 150, 64, 110, 116, 131, 69, 135, 125, 62, 116, 82, 82, 95, 192, 138, 126, 71, 86, 44, 57, 91, 83, 92, 153, 92, 122, 89, 170, 84, 61, 65, 113, 125, 96, 115, 82, 70, 120, 182, 174, 139, 124, 183, 65, 151, 79, 155, 178, 117, 147, 99, 56, 154, 68, 179, 81, 46, 79, 69, 93, 110, 56, 140, 123, 117, 136, 132, 83, 126, 151, 95, 75, 68, 74, 152, 141, 125, 61, 63, 110, 107, 48, 120, 82, 84, 84, 65, 65, 142, 85, 120, 206, 87, 53, 124, 181, 173, 50, 162, 91, 162, 83, 93, 169, 53, 245, 213, 100, 81, 110, 151, 76, 132, 97, 64, 159, 108, 77, 78, 164, 94, 110, 148, 121, 103, 65, 54, 116, 190, 152, 182, 182, 87, 152, 80, 103, 104, 49, 44, 128, 79, 80, 93, 170, 127, 77, 99, 115, 115, 111, 165, 146, 79, 70, 51, 74, 187, 71, 88, 143, 65, 78, 93, 67, 172, 94, 135, 97, 92, 147, 70, 114, 118, 51, 164, 155, 65, 63, 77, 116, 66, 69, 67, 157, 75, 121, 92, 141, 185, 183, 104, 89, 111, 111, 61, 63, 151, 75, 106, 111, 90, 86, 68, 173, 88, 175, 188, 176, 62, 93, 59, 113, 80, 55, 94, 71, 79, 130, 181, 110,

In [ ]:
test.astype(object)

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","[O, O, O, O, B-название проекта, I-название пр..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"[O, O, O, O, O, O, O, O, O, O, O, O, B-видеоиг..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"[O, O, O, O, O, O, B-видеоигра, I-видеоигра, O..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"[O, O, O, O, B-персона, I-персона, O, O, O, O,..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"[O, O, O, O, B-организация, I-организация, O, ..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"[O, O, O, O, B-персона, I-персона, O, O, O, O,..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"[O, O, O, O, O, B-персона, I-персона, O, O, O,..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"[O, O, O, O, B-персона, I-персона, O, O, O, O,..."


In [ ]:
test.to_csv("submit_wiki_v2.csv", index=False)

In [ ]:
pd.read_csv("submit_wiki_v2.csv")

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","['O', 'O', 'O', 'O', 'B-название проекта', 'I-..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"['O', 'O', 'O', 'O', 'O', 'O', 'B-видеоигра', ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"['O', 'O', 'O', 'O', 'B-персона', 'I-персона',..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"['O', 'O', 'O', 'O', 'B-организация', 'I-орган..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"['O', 'O', 'O', 'O', 'B-персона', 'I-персона',..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"['O', 'O', 'O', 'O', 'O', 'B-персона', 'I-перс..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"['O', 'O', 'O', 'O', 'B-персона', 'I-персона',..."


In [ ]:
pd.read_csv("sample_submission.csv")

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
